In [104]:
import numpy as np
import pandas as pd

### $C = \cfrac{ \text{Number of concordant pairs}}{\text{Number of all pairs}} $

## $C =  \frac{\sum_{i, j} 1_{T_{j}<T_{i}} \cdot 1_{\eta_{j}>\eta_{i}} \cdot \delta_{j}}{\sum_{i, j} 1_{T_{j}<T_{i}} \cdot \delta_{j}} $ 

$\eta_{j}$ is the risk score of patient i

$1_{T_{j}<T_{i}}=1 \text { if } T_{j}<T_{i} \text { else } 0$

$1_{\eta_{j}>\eta_{i}}=1 \text { if } \eta_{j}>\eta_{i} \text { else } 0$

## Time, risk score, censoring

In [2]:
n = 10 #number of subjects

In [78]:
T = np.array([i+1 for i in range(n)])

In [79]:
δ = np.array([1. for i in range(n)])

In [80]:
η = 1./T

### Values:

In [81]:
T

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [83]:
η

array([1.        , 0.5       , 0.33333333, 0.25      , 0.2       ,
       0.16666667, 0.14285714, 0.125     , 0.11111111, 0.1       ])

In [82]:
δ

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

### Formula 2 code:

## $C =  \frac{\sum_{i, j} 1_{T_{j}<T_{i}} \cdot 1_{\eta_{j}>\eta_{i}} \cdot \delta_{j}}{\sum_{i, j} 1_{T_{j}<T_{i}} \cdot \delta_{j}} $ 

$1_{T_{j}<T_{i}}=1 \text { if } T_{j}<T_{i} \text { else } 0$:

In [343]:
def δT(Ti, Tj): 
    if Tj < Ti: return 1. 
    else: return 0.  

$1_{\eta_{j}>\eta_{i}}=1 \text { if } \eta_{j}>\eta_{i} \text { else } 0$

In [344]:
def δη(ηi, ηj): 
    if ηj > ηi: return 1. 
    else: return 0. 

In [345]:
def c_index(T, η, δ):
    
    n = len(T)
    numer = 0
    denom = 0
    
    for i in range(n):
        for j in range(n):
            
            numer += δT(T[i], T[j]) * δη(η[i], η[j]) * δ[i]
            
            denom += δT(T[i], T[j]) * δ[i]
            
    return numer/denom

In [346]:
c_index(T, η, δ)

1.0

In [146]:
c_index(T, T, δ)

0.0

## Use time instead of risk score:

In [347]:
def c_index(T, Tpred, δ):
    
    n = len(T)    
    numer = 0
    denom = 0
    
    for i in range(n):
        for j in range(n):
            
            numer += δT(T[i], T[j]) * δT(Tpred[i], Tpred[j]) * δ[i]
            
            denom += δT(T[i], T[j]) * δ[i]
            
    return numer/denom

In [101]:
c_index(T, T, δ)

1.0

In [102]:
c_index(T, 1/T, δ)

0.0

## Interpretation

In [118]:
patients =    ['Alice', 'Bob', 'Charlie', 'Dave', 'Eve']
Time_true =   np.array([1, 2, 3, 4, 5])
Time_pred   = np.array([1, 2, 3, 4, 5])

df = pd.DataFrame(data={'True times': Time_true, 'Predicted': Time_pred}, index=patients)
df

,True times,Predicted
Alice,1,1
Bob,2,2
Charlie,3,3
Dave,4,4
Eve,5,5


In [119]:
c_index(Time_true, Time_pred, δ)

1.0

### This is true for any strictly increasing function 

In [120]:
patients =    ['Alice', 'Bob', 'Charlie', 'Dave', 'Eve']
Time_true =   np.array([1, 2, 3, 4, 5])
Time_pred   = np.exp(np.array([1, 2, 3, 4, 5]))

df = pd.DataFrame(data={'True times': Time_true, 'Predicted': Time_pred}, index=patients)
df

,True times,Predicted
Alice,1,2.718282
Bob,2,7.389056
Charlie,3,20.085537
Dave,4,54.598150
Eve,5,148.413159


In [122]:
c_index(Time_true, Time_pred, δ)

1.0

## Completely false order:

In [123]:
patients =    ['Alice', 'Bob', 'Charlie', 'Dave', 'Eve']
Time_true =   np.array([1, 2, 3, 4, 5])
Time_pred   = np.array([5, 4, 3, 2, 1])

df = pd.DataFrame(data={'True times': Time_true, 'Predicted': Time_pred}, index=patients)
df

,True times,Predicted
Alice,1,5
Bob,2,4
Charlie,3,3
Dave,4,2
Eve,5,1


In [124]:
c_index(Time_true, Time_pred, δ)

0.0

## Order of the predictions is neither completely right nor completely wrong

In [125]:
patients =    ['Alice', 'Bob', 'Charlie', 'Dave', 'Eve']
Time_true =   np.array([1, 2, 3, 4, 5])
Time_pred   = np.array([3, 2, 1, 5, 4])

df = pd.DataFrame(data={'True times': Time_true, 'Predicted': Time_pred}, index=patients)
df

,True times,Predicted
Alice,1,3
Bob,2,2
Charlie,3,1
Dave,4,5
Eve,5,4


In [126]:
c_index(Time_true, Time_pred, δ)

0.6

### $C = \cfrac{ \text{Number of concordant pairs}}{\text{Number of all pairs}} $

Number of all possible pairs = n(n-1)/2 = 10

#### Not noncordant pairs:

Alice-Bob

Alice-Charlie

Bob-Charlie

Dave-Eve

-> 10 - 4 = 6 concordant pairs 

### MULTIVARIABLE PROGNOSTIC MODELS: ISSUES IN DEVELOPING MODELS, EVALUATING ASSUMPTIONS AND ADEQUACY, AND MEASURING AND REDUCING ERRORS FRANK E. HARRELL Jr., KERRY L. LEE AND DANIEL B. MARK

"When predicted survivals are identical for a patient pair,
1/2 rather than 1 is added to the count of concordant pairs in the
numerator of c."

In [363]:
patients =    ['Alice', 'Bob', 'Charlie', 'Dave', 'Eve']
Time_true =   np.array([1., 2., 3., 4., 5.])
Time_pred   = np.array([1., 2., 3., 4., 4.])

df = pd.DataFrame(data={'True times': Time_true, 'Predicted': Time_pred}, index=patients)
df

,True times,Predicted
Alice,1.0,1.0
Bob,2.0,2.0
Charlie,3.0,3.0
Dave,4.0,4.0
Eve,5.0,4.0


In [364]:
def δT(Ti, Tj): 
    if Tj < Ti: 
        return 1.
    else: 
        return 0.  

In [365]:
def δTties(Ti, Tj): 
    if Tj < Ti: 
        return 1. 
    if Tj == Ti: 
        return 0.5
    else: 
        return 0.  

In [366]:
def c_index(T, Tpred, δ):
    
    n = len(T)
    numer = 0
    denom = 0
    
    for i in range(n):
        for j in range(n):
            
            numer += δT(T[i], T[j]) * δTties(Tpred[i], Tpred[j]) * δ[i]
            
            denom += δT(T[i], T[j]) * δ[i]
            
    return numer/denom

In [367]:
c_index(Time_true, Time_pred, δ)

0.95

### In case of censoring

In [371]:
patients  =    ['Alice', 'Bob', 'Charlie', 'Dave', 'Eve']
Time_true =   np.array([1., 2., 3., 4., 5.])
Time_pred =   np.array([1., 2., 3., 5., 4.])
Observed  =   np.array([1, 1, 1, 1, 1])

df = pd.DataFrame(data={'True times': Time_true, 'Predicted': Time_pred, 'Observed': Observed}, index=patients)
df

,True times,Predicted,Observed
Alice,1.0,1.0,1
Bob,2.0,2.0,1
Charlie,3.0,3.0,1
Dave,4.0,5.0,1
Eve,5.0,4.0,1


In [372]:
c_index(Time_true, Time_pred, δ)

0.9

In [373]:
patients  =    ['Alice', 'Bob', 'Charlie', 'Dave', 'Eve']
Time_true =   np.array([1., 2., 3., 4., 5.])
Time_pred =   np.array([1., 2., 3., 5., 4.])
Observed  =   np.array([1, 1, 0, 1, 1])

df = pd.DataFrame(data={'True times': Time_true, 'Predicted': Time_pred, 'Observed': Observed}, index=patients)
df

,True times,Predicted,Observed
Alice,1.0,1.0,1
Bob,2.0,2.0,1
Charlie,3.0,3.0,0
Dave,4.0,5.0,1
Eve,5.0,4.0,1


Number of all possible pairs = n(n-1)/2 = 10

Charlie censored at 3 years -> C-D and C-E not counted -> 10-2 = 8 pairs

D-E and E-D not concordant -> 8-1 = 7 condordant pairs

-> c-index = 7/8 = 0.875

In [376]:
c_index(Time_true, Time_pred, Observed)

0.875